In [2]:
from AlgorithmicTrading.strategies.perceptron import Strategy
from AlgorithmicTrading.account import AccountBacktest, AccountLive
from AlgorithmicTrading.models.metatrader import ENUM_ACCOUNT_MARGIN_MODE, ENUM_TIMEFRAME
from AlgorithmicTrading.rates import Rates
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import MetaTrader5 as mt5

In [3]:
# Netting account
account = AccountLive.login(login=5013526569, server="MetaQuotes-Demo", password="h3xhhvuo")
# Backtest account
account = AccountBacktest.login(balance=100_000, margin_mode=ENUM_ACCOUNT_MARGIN_MODE.ACCOUNT_MARGIN_MODE_RETAIL_NETTING)
account

[INFO]: Successfull login to #5013526569 account
[INFO]: Successfull backtest account create


MqlAccountInfo(login=9999999, trade_mode=<ENUM_ACCOUNT_TRADE_MODE.ACCOUNT_TRADE_MODE_DEMO: 0>, leverage=100, limit_orders=200, margin_so_mode=<ENUM_ACCOUNT_STOPOUT_MODE.ACCOUNT_STOPOUT_MODE_PERCENT: 0>, trade_allowed=True, trade_expert=True, margin_mode=<ENUM_ACCOUNT_MARGIN_MODE.ACCOUNT_MARGIN_MODE_RETAIL_NETTING: 0>, currency_digits=2, fifo_close=False, balance=100000.0, credit=0.0, profit=0.0, equity=100000.0, margin=0.0, margin_free=100000.0, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Backtest Name', server='Backtest Server', currency='USD', company='Backtest Company', orders=[], positions=[], history_deals=[], is_backtest_account=True)

In [4]:
n_rates = Rates.get_last_n_candles("USDJPY", timeframe=ENUM_TIMEFRAME.TIMEFRAME_H1).reset_index()
n_rates

,time,open,high,low,close,tick_volume
0,2021-12-15 07:00:00+00:00,113.768,113.770,113.712,113.727,1262
1,2021-12-15 08:00:00+00:00,113.726,113.742,113.628,113.671,1419
2,2021-12-15 09:00:00+00:00,113.670,113.744,113.646,113.726,3061
3,2021-12-15 10:00:00+00:00,113.726,113.758,113.652,113.696,3905
4,2021-12-15 11:00:00+00:00,113.697,113.751,113.665,113.743,2950
...,...,...,...,...,...,...
9995,2023-07-21 19:00:00+00:00,141.747,141.767,141.623,141.670,1831
9996,2023-07-21 20:00:00+00:00,141.671,141.722,141.632,141.716,1802
9997,2023-07-21 21:00:00+00:00,141.716,141.813,141.678,141.808,1648
9998,2023-07-21 22:00:00+00:00,141.809,141.827,141.723,141.760,1074


In [5]:
features = Strategy.features_engineering(n_rates).dropna()
features

,open,high,low,close,tick_volume,Target - returns,Feature - 5 SMA,Feature - 10 SMA,Feature - 20 SMA,Feature - 30 SMA,...,Feature - 21 RSI SMA,Feature - MACD Line,Feature - MACD Signal Line,Feature - MACD Diff,Feature - Bollinger Bands High Band,Feature - Bollinger Bands Mid Band,Feature - Bollinger Bands Low Band,Feature - Bollinger Bands P Band,Feature - Bollinger Bands W Band,Feature - Acc/Dist Index
33,114.127,114.127,113.656,113.678,9143,0.071,113.990476,114.051313,114.053969,114.038660,...,60.999951,0.034822,0.075793,-0.040971,114.306298,114.09275,113.879202,-0.471093,0.374341,-1.998477e+03
34,113.676,113.779,113.562,113.747,7599,-0.074,113.909317,113.995934,114.023826,114.017825,...,57.798784,0.007574,0.062149,-0.054575,114.339302,114.07905,113.818798,-0.137940,0.456266,3.359343e+03
35,113.749,113.758,113.596,113.675,4901,-0.040,113.831211,113.937540,113.989674,113.993502,...,54.743788,-0.019605,0.045798,-0.065403,114.374924,114.06065,113.746376,-0.113557,0.551065,3.238331e+03
36,113.674,113.694,113.609,113.634,2590,0.047,113.765474,113.882318,113.954944,113.968160,...,51.976860,-0.043945,0.027850,-0.071795,114.405389,114.04010,113.674811,-0.055861,0.640632,2.171860e+03
37,113.636,113.706,113.599,113.683,1826,-0.081,113.737983,113.846061,113.928454,113.948178,...,50.060143,-0.058606,0.010558,-0.069165,114.419990,114.02285,113.625710,0.072129,0.696598,3.212851e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,141.750,141.772,141.521,141.747,5357,-0.077,141.645980,141.418819,141.001937,140.711620,...,67.351281,0.507205,0.429998,0.077206,142.388272,140.79490,139.201528,0.798769,2.263394,3.797426e+06
9995,141.747,141.767,141.623,141.670,1831,0.045,141.653987,141.464489,141.065562,140.773451,...,67.340195,0.497259,0.443450,0.053809,142.475051,140.87515,139.275249,0.748406,2.271374,3.796790e+06
9996,141.671,141.722,141.632,141.716,1802,0.092,141.674658,141.510218,141.127509,140.834260,...,67.390669,0.487470,0.452254,0.035216,142.550844,140.95880,139.366756,0.737808,2.258878,3.798352e+06
9997,141.716,141.813,141.678,141.808,1648,-0.049,141.719105,141.564360,141.192317,140.897082,...,67.555353,0.481585,0.458120,0.023464,142.612522,141.05250,139.492478,0.742144,2.211974,3.799878e+06


In [6]:
features.columns

Index(['open', 'high', 'low', 'close', 'tick_volume', 'Target - returns',
       'Feature - 5 SMA', 'Feature - 10 SMA', 'Feature - 20 SMA',
       'Feature - 30 SMA', 'Feature - 50 SMA', 'Feature - 100 SMA',
       'Feature - 200 SMA', 'Feature - 250 SMA', 'Feature - 14 RSI',
       'Feature - 21 RSI', 'Feature - 14 RSI SMA', 'Feature - 21 RSI SMA',
       'Feature - MACD Line', 'Feature - MACD Signal Line',
       'Feature - MACD Diff', 'Feature - Bollinger Bands High Band',
       'Feature - Bollinger Bands Mid Band',
       'Feature - Bollinger Bands Low Band',
       'Feature - Bollinger Bands P Band', 'Feature - Bollinger Bands W Band',
       'Feature - Acc/Dist Index'],
      dtype='object')

In [7]:
train_set, test_set = train_test_split(features, test_size=0.2, random_state=42)

In [8]:
selected_features = ['open', 'high', 'low', 'close', 'tick_volume',
       'Feature - 5 SMA', 'Feature - 10 SMA', 'Feature - 20 SMA',
       'Feature - 30 SMA', 'Feature - 50 SMA', 'Feature - 100 SMA',
       'Feature - 200 SMA', 'Feature - 250 SMA', 'Feature - 14 RSI',
       'Feature - 21 RSI', 'Feature - 14 RSI SMA', 'Feature - 21 RSI SMA',
       'Feature - MACD Line', 'Feature - MACD Signal Line',
       'Feature - MACD Diff', 'Feature - Bollinger Bands High Band',
       'Feature - Bollinger Bands Mid Band',
       'Feature - Bollinger Bands Low Band',
       'Feature - Bollinger Bands P Band', 'Feature - Bollinger Bands W Band',
       'Feature - Acc/Dist Index']
X_train = train_set[selected_features].values
X_test = test_set[selected_features].values
X_train

array([[1.42995000e+02, 1.43054000e+02, 1.42624000e+02, ...,
        5.73553936e-01, 6.68827183e-01, 2.07773690e+06],
       [1.46524000e+02, 1.46632000e+02, 1.46344000e+02, ...,
        9.75215765e-01, 7.62198959e-01, 2.55690867e+06],
       [1.38155000e+02, 1.38255000e+02, 1.38118000e+02, ...,
        6.83096194e-01, 2.22997738e-01, 1.67276106e+06],
       ...,
       [1.45721000e+02, 1.46006000e+02, 1.45625000e+02, ...,
        3.44043761e-01, 9.88502328e-01, 2.85578480e+06],
       [1.14915000e+02, 1.15393000e+02, 1.14863000e+02, ...,
        1.41345803e+00, 3.74933331e-01, 2.04329259e+05],
       [1.33366000e+02, 1.33397000e+02, 1.33207000e+02, ...,
        8.54283623e-01, 6.40646411e-01, 2.97548802e+06]])

In [9]:
y_train = train_set["Target - returns"] > 0
y_test = test_set["Target - returns"] > 0
y_train

4630    False
5162     True
3721    False
3754    False
4567    False
        ...  
5767    False
5224     True
5423     True
893     False
7303     True
Name: Target - returns, Length: 7972, dtype: bool

In [10]:
model = Perceptron(random_state=42)
pipeline = make_pipeline(StandardScaler(), model)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('perceptron', Perceptron(random_state=42))])

In [11]:
y_pred = pipeline.predict(X_test)
str(round(accuracy_score(y_pred=y_pred, y_true=y_test) * 100)) + "%"


'51%'